# Basics of parallel and distributed computing with DASK

## Recap on task parallelization (this time using Dask)

We should now start to feel quite familiar with the programming pattern already discussed in the previous lectures when we discussed Hadood MapReduce and the Spark framework.

Asides from the details of the API, which depend on the framework implementation, and from the inner structure of the job scheduling and resorce manager, most of the distributed computing frameworks offer us a way of parallelizing our jobs by means of Graph schedulers and Task optimizers.

The goal is always the same, no matter the tool used to reach it:
- store large datasets which won't fit into memory in a set of partitions 
- first plan the entire data processing, then optimize it to break it into smaller tasks
- finally, distribute the processing close to the data, instead of moving the data (data-locality)

In Dask, we still have access to a set of dataset representations:
- Dask bag (~ equivalent to the Spark RDD)
- Dask array
- Dask dataframe (~ equivalent to the Spark DataFrame)

However, Dask is mainly a scheduler (this time written in Python and not in Scala as Spark \[*\]), which allows lazy execution of python-like code for distributing it across a multitude of workers.


\[*\] To be clear, the fact that the scheduler is in Python does not mean it's _better_ than Spark for our purposes.  Python and Scala/Java are different in a multitude of ways, and both have pros and cons. For instance, as of today, it is known that due to the way Python handles and manages memory during the execution of an application, Dask still suffers from a issues with so-called unmanaged memory throughout the life of an application (see for instance the [link](https://coiled.io/blog/tackling-unmanaged-memory-with-dask/)).

## Starting a cluster and first lazy computation with `delayed`

We can use Dask locally, to exploit the multi-tasking/processing functionalities, or setup a cluster and deploy a scheduler and a number of worker nodes.

After we setup a cluster, we initialize a Client by pointing it to the address of a Scheduler.

In [ ]:
# set this variable with one of the following values

# -> 'local'
# -> 'docker_container'
# -> 'docker_cluster'

CLUSTER_TYPE ='docker_cluster'

In [ ]:
%env CLUSTER_TYPE $CLUSTER_TYPE

In [ ]:
%%script bash --bg --out script_out

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    echo "Launching scheduler and worker"
    
    HOSTIP=`hostname -I | xargs`
    
    echo "dask-scheduler --host $HOSTIP --dashboard-address $HOSTIP:8787"
    
    # dask scheduler 
    dask-scheduler --host $HOSTIP --dashboard-address $HOSTIP:8787 &

    # dask worker
    dask-worker $HOSTIP:8786 --memory-limit 2GB --nworkers 2 &

fi

In [ ]:
host_ip = !hostname -I | xargs
host_ip = host_ip[0]

In [ ]:
from dask.distributed import Client

if CLUSTER_TYPE == 'local':
    
    client = Client()

elif CLUSTER_TYPE == 'docker_container':
    
    client = Client('{}:8786'.format(host_ip))
    
elif CLUSTER_TYPE == 'docker_cluster':
    
    # use the provided master
    client = Client('dask-scheduler:8786')
    
client

We can check the status of the cluster at the `localhost:8787` dashboard, where we have an overview of both the status of the workers, and the execution of the DAG task graphs, when we'll start using them.

The simplest example to parallelize any arbitrary Python code in Dask can be presented by running a couple of simple arbitrary operarions, described here with 2 functions and an arbitrary sleep time of 1 second.
The sleep time is meant to represent an arbitrariliy complex code, and the time required for its execution.

In [ ]:
from time import sleep

def increment(x):
    # sleep for 1s
    sleep(1)
    
    # given the input x, return x+1
    return x + 1

def decrement(x):
    # sleep for 1s
    sleep(1)

    # given the input x, return x-1
    return x - 1


def add(x, y):
    # sleep for 1s
    sleep(1)
    
    # given the inputs x and y, return x+y
    return x + y

We can test the functions locally by running them on the client (not on the cluster).

In [ ]:
%%time

x = increment(1)
y = decrement(2)
z = add(x, y)

In order to let Dask take advantage of the processing units assigned to the cluster, we have to let it elaborate the DAG corresponding to the execution of the code we want to deploy on the cluster, and then let the scheduler dispatch the tasks to the workers.

This is done in Dask by declaring that a function is `delayed`.

The `delayed` Dask method takes several arguments. 
The first argument is the function that has to be executed in parallel.
The following arguments are the arguments of the original function.

We now want to transform the `inc` and `add` functions, therefore making them `lazy`.

In [ ]:
from dask import delayed

In [ ]:
%%time

x = delayed(increment)(1)
y = delayed(decrement)(2)
z = delayed(add)(x, y)

At this stage, as usual with lazy operations, we don't have the results stored in `z`.

`z` at this stage is just the "plan" of the code execution, made by the DAG task scheduler.

We can visualize the plan of the execution with the `visualize()` method.

In [ ]:
z.visualize()

In order to actually execture the job, we have to ask Dask to return the results of the lazy operation with the `compute()` method.

Under the carpet, Dask will ship the computational graph to the scheduler, and will dispatch the tasks to the workers, similarly to what already discussed for Spark

In [ ]:
%%time

z.compute() 

## Futures and Eager computation with `submit`

The eager operation alternative to `delayed` in Dask is the `submit`, which instruct Dask to start executing our task on the cluster right away.

In [ ]:
future = client.submit(increment, 1)

The function `increment` with its agurment `1` is submitted to the cluster, and the `future` object is returned right away. 

The `submit` operation has in fact returned a so-called execution *promise* (the `future` variable) of the instruction that we have submitted.
It doesn't necessarily mean that the execution has been completed, as the cluster might take some time to execute it.

The `future` variable in fact **doen not contains the result**, but just the promise of it, when the elaboration will be completed. 
The result of the computation will be left on the worker nodes of our cluster, and will not be sent back to our client until we reclaim it.

To retrieve the result we do invoke the `gather` function (similarly to the collect in Spark).

In [ ]:
future

In [ ]:
client.gather(future)

This approach can be extremely useful in situations where we need to submit an instruction multiple times, perhaps with different input parameters. 
This is a for instance a quite frequent scenario in in machine learning, for example when we need to optmize an algorithm for a particular dataset over the hyper-parameter space. 

The idea of this approach allows us to `map` the instruction that we want to executo to each argument of a dataset, thus submitting the same operation on the entire dataset.

In [ ]:
data = [0, 1, 2, 3, 4, 5, 6, 7, 8]

future_results = client.map(increment, data)
future_results

In [ ]:
new_data = client.gather(future_results)
print(new_data)

If we need to wait for the result of a `submit` computation to be ready (say we need it as an input in other computations), we may still ask Python to block the execution of the new code and `wait` the computation of the future. 

In [ ]:
from dask.distributed import wait

new_future = client.map(increment, new_data)
wait(new_future)

In [ ]:
client.gather(new_future)

In complete analogy to what we have discussed with the `delayed` lazy execution, we can combine multiple instruction that have to be submitted to the cluster and create a more complex job to run on our cluster.

We should remember that the results of the `submit` execution reside on the cluster until a `gather` is used.

This means that we can submit to the cluster a task that takes as argument an execution promise of an instruction that has been previosly submitted. 

The `gather` function can thus be invoked at the end of the program, invoked only when the results have to be effectively retrived from the cluster.

In [ ]:
x = client.submit(increment, 1)
y = client.submit(decrement, 2)
total = client.submit(add, x, y)

In [ ]:
print(total)          # This is still the execution promise

In [ ]:
client.gather(total)  # This is the final result

### Exercise: Parallelization of a for loop

At this point, with the previous knowledge of what discussed with Spark, and the `delayed` and `compute` Dask operations, we should already be able to run simple "dummy" tasks.

Starting from a list $\vec{x}$ of numbers:
1. increment each element $x_i$ by a random value $\delta x_i$ (in the 0-1 range)
2. multiply the resulting value by 10
3. sum all resulting values

In [ ]:
import time

data = [1, 2, 3, 4, 5, 6, 7, 8]

import random

# increment function
def add_rand(x):
    ...

# multiplication function
def mult_ten(x):
    ...

start = time.time()

results = ...

for x in data:
    y =  ... # put your code here.
    z =  ... # put your code here.
    results. ... # put your code here.
    
total = ... #put your code here. 

end = time.time()

In [ ]:
total.visualize()

In [ ]:
%%time
result = total.compute()

In [ ]:
print("result: ",result)

Run the same task with the eager `submit` execution

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

results = []

for x in data:
    y =  ... # put your code here.
    z =  ... # put your code here.
    results. ... # put your code here.
    
total = ... #put your code here. 

In [ ]:
total

In [ ]:
print("result: ",result)

There is clearly another alternative to the previous approach:

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

y =  ... # put your code here.
z =  ... # put your code here.
    
total = ... #put your code here. 

In [ ]:
total

In [ ]:
print("result: ",result)

### Exercise: Sum reduction on a tree

So far we acted mostly with _map_ like operations, and we collected all outputs in a single `sum` operation.

We might want to briefly explore how we could write in Dask an equivalent _reduce_ function to evaluate the sum of a list of elements pair-wise

Here there is a schema on wath we have to do:

```
finish           total             single output
    ^          /        \
    |        c1          c2        neighbors merge
    |       /  \        /  \
    |     b1    b2    b3    b4     neighbors merge
    ^    / \   / \   / \   / \
start   a1 a2 a3 a4 a5 a6 a7 a8    many inputs
```

Here there is the usual pair-reduction algorithm with nested for loop and a bit of simple Python logic.

In [ ]:
L = [1, 2, 3, 4, 5, 6, 7, 8, 9]

start = time.time()
while len(L) > 1:
    _ = []
    for i in range(0, len(L), 2):
        if i+1 < len(L):
            pair_sum = add(L[i], L[i + 1])  # add neighbor
        else:
            pair_sum = add(L[i], 0)         # add 0
        _.append(pair_sum)
    L = _ 
    
print("result:",L[0])
end = time.time()

print("Computation took {:.2f}s".format(end-start))

To parallelize this reduce task, we define as `delayed` the pair-wise sum of neighbour elements.

In [ ]:
L = [1, 2, 3, 4, 5, 6, 7, 8, 9]

while len(L) > 1:
   ... # put your code here.

In [ ]:
L[0].visualize()

In [ ]:
%%time 
result = L[0].compute()

In [ ]:
print("result",result)

### Exercise: Count how many words are present in a series of documents 

Starting from a dataset of pieces of text taken from `sklearn` we would like to count how many words are present in each document and calculate how many words are in the total dataset. 

The documents are ~8000.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import time

categories = [
     'comp.graphics',
     'comp.os.ms-windows.misc',
     'comp.sys.ibm.pc.hardware',
     'comp.sys.mac.hardware',
     'comp.windows.x',
     'misc.forsale',
     'rec.autos',
     'rec.motorcycles',
     'rec.sport.baseball',
     'rec.sport.hockey',
     'sci.crypt',
     'sci.electronics',
     'sci.med',
     'sci.space'
]

dataset = fetch_20newsgroups(subset='train', categories=categories ).data

In [ ]:
print("Documents in the dataset:",len(dataset))

In [ ]:
def count_words_in_document(text):
    ... # put your code here.

The single-threaded execution in Python can be run by the following

In [ ]:
%%time

total_words_in_all_data = 0
total_words_in_document = []

for index in range(len(dataset)):
    total_words_in_document. ... # put your code here.
    total_words_in_all_data = ... # put your code here.

In [ ]:
print("Total number of words in the dataset: {}".format(total_words_in_all_data))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(total_words_in_document,bins=range(0,1000,10));
plt.xlabel('words per document');
plt.ylabel('counts');

Distributed version using `map` and `submit`:

In [ ]:
%%time

total_words_in_document = ...
total_words_in_all_data = ...

In [ ]:
result = ...
print("Total number of words in the dataset: {}".format(result))

### Exercise: Calculate the first $n$ Fibonacci numbers

Let's define the algorithm to evaluate the sequence of Fibonacci up to the $n$-th element:

1,1,3,5,8,13,...

In [ ]:
def fibonacci_sequential(num):
    i = 1
    if num <= 0:
        ... # put your code here.
    elif num == 1:
        ... # put your code here.
    elif num == 2:
        ... # put your code here.
    elif num > 2:
        ... # put your code here.
        while i < (num - 1):
            ... # put your code here.
            i += 1
    return fibonacci

result = fibonacci_sequential(8)
print("The first n fibonacci numbers are: " + str(result))

Let's implement the same algorithm using the Dask lazy evaluation, with `delayed`, and inspect the task graph

In [ ]:
# in order to generalize the append
def append(arr = [], val = 0):
    if val != None:
        arr.append(val)
    return arr

def fibonacci(num):
    i = 1
    if num == 0:
        ... # put your code here
    elif num == 1:
        ... # put your code here.
    elif num == 2:
        ... # put your code here.
    elif num > 2:
        ... # put your code here
        while i < (num - 1):
            ... # put your code here.
            i += 1
    return ...  # put your code here
result = fibonacci(8)
result.visualize(rankdir="LR")

In [ ]:
result.compute()

### Exercise: Monte Carlo Integration

Let's assume we want to integrate of a function via MonteCarlo technique, as you have discussed in LCP Module A.

Let's use the function $$f(x) =\sin^2{\frac{1}{x(2-x)}}$$ and let's integrate in the range $(0,2)$

In [ ]:
import numpy as np

def f(x):
    return (np.sin(1/(x*(2-x))))**2

x=np.linspace(-0.2,2.2,1000)

plt.figure(figsize=(16,6));
plt.plot(x,f(x),'grey','.');
plt.fill_between(x[np.where((x>0) & (x<2))],[1]*len(np.where((x>0) & (x<2))), alpha=0.2);
plt.fill_between(x[np.where((x>0) & (x<2))],f(x[np.where((x>0) & (x<2))]), alpha=0.2);
plt.vlines([0, 2], 0, 1, colors = ["k", "k"], linestyles = ["dashed", "dashed"],linewidths=[3,3],zorder=20);
plt.xlabel('x');
plt.ylabel('$f(x)$');

Create the single-thread Python code to execute this task, and evaluate the integral over N=100 000 points

In [ ]:
%%time
# Monte Carlo integration
N=100000
count=[]
def pass_function():
    x=2*np.random.random()
    y=np.random.random()
    return 1 if y<f(x) else 0

for i in range(N):
    count.append(pass_function())

I=2*sum(count)/N
print("Integral=",I)

Apply the minimal changes to the code to deploy the same integral computation on the Dask cluster

**NOTE**: Do NOT use 100 000 points in this case, but _limit the computation to N=1000 points_

In [ ]:
# Monte Carlo integration
N=1000

... # put your code here

real_count = ...     # put your code here

In [ ]:
%%time
I=2*real_count.compute()/N

In [ ]:
print("Integral=",I)

What is the reason why we are getting worse performance with respect to the single threaded execution?

### Exercise: $\pi$ via MonteCarlo

You can re-run the evaluation of $\pi$ from the MonteCarlo technique, as done during the Spark hands-on session.

Just be careful that so far with Dask we are not creating data partitions _just yet_.

We are instructing Dask to run a simple task per each entry in our list, thus results in a strong overhead.

For this reason, use a very limited number of points (start with ~10 and max ~ 100), and check the status of the job from the dashboard.

In [ ]:
# plain python - single threaded
%%time 

num_points = 10000

points_in_circle = []

def in_circle():
    ## simulate the point and check if
    ## it is inside the circle 
    ## return 0 or 1
    ... # put your code here

for p in range(num_points):
    ... # put your code here

num_points_inside = ... # put your code here

In [ ]:
# print result
print ("pi =", 4*num_points_inside/num_points)

With `delayed`

In [ ]:
%%time 
import random

num_points = 100

... # put your code here

points_inside = ... # put your code here

In [ ]:
# points_inside.visualize()

In [ ]:
%%time
num_points_inside = points_inside.compute()

In [ ]:
# print result
print ("pi =", 4*num_points_inside/num_points)

With `submit` we have to be very careful...
Calling naively `client.map` with a function returning a random variable would imply returning all the times the same exact random value (we have 1 function mapped on all the data).

We can however instruct Dask that this function is in fact `inpure`, namely, is a function that does not really returns the same result for every execution.

Thus:

In [ ]:
%%time 

... # put your code here

points_in_circle = client.map(... """ put your code here """, pure=False) # pure=False is needed to use the map function in this case

... # put your code here

In [ ]:
# print result
print ("pi =", 4*num_points_inside/num_points)